In [5]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024                 # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 48000                 # samples per second

In [16]:
# Create matplotlib figure and axes
fig, ax = plt.subplots()

# Create a PyAudio class instance
p = pyaudio.PyAudio()

# Stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# Variables for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# Create a line object with random data - cuz streaming
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# Semilogx line for spectrum? Check if better idea
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# Format plot axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('Samples')
ax.set_ylabel('Volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])


# Measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    # Binary data
    data = stream.read(CHUNK)  
    
    # Convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # Create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # Compute FFT and update line -> improve here
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    
    # Update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # Calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('average frame rate = {:.0f} FPS'.format(frame_rate)) # can improve matplot plotting -> is there non matplot for audio??
        break

average frame rate = 15 FPS
